In [1]:

# Import Library
# text preprocessing
import pandas as pd
import numpy as np
import csv
import requests
import io
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import re # regular expression
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory # stemming indonesian language
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from nltk.corpus import stopwords
from tqdm import tqdm

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer # to create Bag of words
from sklearn.feature_extraction.text import TfidfVectorizer # tfid Vector 
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score # confussion matrix
from sklearn.preprocessing import LabelEncoder # to convert classes to number 
from sklearn.model_selection import train_test_split  # for splitting data 
from sklearn.metrics import accuracy_score # to calculate accuracy
from sklearn.pipeline import Pipeline
#from mlxtend.plotting import plot_confusion_matrix

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
dataset = pd.read_csv(r'before_preprocessing.csv')

In [3]:
dataset.head()

,review_id,review_text,price,packaging,product,aroma
0,708,sunscreen termahal yang pernah gue beli ini ka...,-1,0,1.0,0
1,838,"ini enak banget dipakainya, enteng banget diwa...",0,0,1.0,0
2,1166,the description is quite right. produk ini ema...,0,0,1.0,1
3,1374,bisa untuk wajah dan badan dengan harga yang s...,1,1,1.0,0
4,1421,saya beli produk ini karena suka banget wangin...,-1,0,0.0,1


In [4]:
dataset.isnull().sum()

review_id      0
review_text    0
price          0
packaging      0
product        1
aroma          0
dtype: int64

In [5]:
dataset = dataset.fillna(method="ffill")
dataset.isnull().sum()

review_id      0
review_text    0
price          0
packaging      0
product        0
aroma          0
dtype: int64

In [6]:
dataset.head()

,review_id,review_text,price,packaging,product,aroma
0,708,sunscreen termahal yang pernah gue beli ini ka...,-1,0,1.0,0
1,838,"ini enak banget dipakainya, enteng banget diwa...",0,0,1.0,0
2,1166,the description is quite right. produk ini ema...,0,0,1.0,1
3,1374,bisa untuk wajah dan badan dengan harga yang s...,1,1,1.0,0
4,1421,saya beli produk ini karena suka banget wangin...,-1,0,0.0,1


In [7]:
# casefolding 

def clean(dataset):
  temp_text = []

  for txt in dataset:
    # removal of @name[mention]
    txt = re.sub(r"(?:\@|https?\://)\S+", "", txt)

    # removal of links[https://blabala.com]
    # tw = re.sub(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", "", tw)
    txt = re.sub(r"http\S+", "", txt)

    # removal of new line
    txt = re.sub('\n', '', txt)

    # removal of RT
    txt = re.sub('RT', '', txt)

    # removal of punctuations and numbers
    txt = re.sub("[^a-zA-Z^']", " ", txt)
    txt = re.sub(" {2,}", " ", txt)

    # remove leading and trailing whitespace
    txt = txt.strip()

    # remove whitespace with a single space
    txt = re.sub(r'\s+', ' ', txt)

    # convert text to Lowercase
    text = txt.lower();
    temp_text.append(txt)
  return temp_text 

dataset['Clean_text'] = clean(dataset['review_text'])
dataset.head()

,review_id,review_text,price,packaging,product,aroma,Clean_text
0,708,sunscreen termahal yang pernah gue beli ini ka...,-1,0,1.0,0,sunscreen termahal yang pernah gue beli ini ka...
1,838,"ini enak banget dipakainya, enteng banget diwa...",0,0,1.0,0,ini enak banget dipakainya enteng banget diwaj...
2,1166,the description is quite right. produk ini ema...,0,0,1.0,1,the description is quite right produk ini eman...
3,1374,bisa untuk wajah dan badan dengan harga yang s...,1,1,1.0,0,bisa untuk wajah dan badan dengan harga yang s...
4,1421,saya beli produk ini karena suka banget wangin...,-1,0,0.0,1,saya beli produk ini karena suka banget wangin...


In [8]:
#TOKENISASI
def token(dataset):
  return dataset.apply(nltk.word_tokenize)

dataset['Clean_text'] = token(dataset['Clean_text'])
dataset.head()

,review_id,review_text,price,packaging,product,aroma,Clean_text
0,708,sunscreen termahal yang pernah gue beli ini ka...,-1,0,1.0,0,"[sunscreen, termahal, yang, pernah, gue, beli,..."
1,838,"ini enak banget dipakainya, enteng banget diwa...",0,0,1.0,0,"[ini, enak, banget, dipakainya, enteng, banget..."
2,1166,the description is quite right. produk ini ema...,0,0,1.0,1,"[the, description, is, quite, right, produk, i..."
3,1374,bisa untuk wajah dan badan dengan harga yang s...,1,1,1.0,0,"[bisa, untuk, wajah, dan, badan, dengan, harga..."
4,1421,saya beli produk ini karena suka banget wangin...,-1,0,0.0,1,"[saya, beli, produk, ini, karena, suka, banget..."


In [9]:
#Stemming

factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stem(dataset) :
    hasil = stemmer.stem(dataset)
    return hasil

dataset['Clean_text'] = dataset['Clean_text'].apply(lambda x: " ".join(x) if isinstance(x, list) else x)
dataset.head()

,review_id,review_text,price,packaging,product,aroma,Clean_text
0,708,sunscreen termahal yang pernah gue beli ini ka...,-1,0,1.0,0,sunscreen termahal yang pernah gue beli ini ka...
1,838,"ini enak banget dipakainya, enteng banget diwa...",0,0,1.0,0,ini enak banget dipakainya enteng banget diwaj...
2,1166,the description is quite right. produk ini ema...,0,0,1.0,1,the description is quite right produk ini eman...
3,1374,bisa untuk wajah dan badan dengan harga yang s...,1,1,1.0,0,bisa untuk wajah dan badan dengan harga yang s...
4,1421,saya beli produk ini karena suka banget wangin...,-1,0,0.0,1,saya beli produk ini karena suka banget wangin...


In [10]:
dataset['Clean_text'] = dataset.apply(lambda row : stem(row['Clean_text']), axis = 1)

In [11]:
dataset.head()

,review_id,review_text,price,packaging,product,aroma,Clean_text
0,708,sunscreen termahal yang pernah gue beli ini ka...,-1,0,1.0,0,sunscreen mahal yang pernah gue beli ini kaya ...
1,838,"ini enak banget dipakainya, enteng banget diwa...",0,0,1.0,0,ini enak banget pakai enteng banget wajah engg...
2,1166,the description is quite right. produk ini ema...,0,0,1.0,1,the description is quite right produk ini eman...
3,1374,bisa untuk wajah dan badan dengan harga yang s...,1,1,1.0,0,bisa untuk wajah dan badan dengan harga yang s...
4,1421,saya beli produk ini karena suka banget wangin...,-1,0,0.0,1,saya beli produk ini karena suka banget wangi ...


In [12]:
slang_word = requests.get('https://raw.githubusercontent.com/louisowen6/NLP_bahasa_resources/master/combined_slang_words.txt').text
dict_slang = eval(slang_word)

df_acronym = pd.read_csv('https://raw.githubusercontent.com/ramaprakoso/analisis-sentimen/master/kamus/acronym.txt', header=None, sep='=')
df_acronym.columns = ['Singkatan', 'kepanjangan']
df_acronym.kepanjangan = df_acronym.kepanjangan.apply(lambda x: x.strip().lower())
df_acronym.Singkatan = df_acronym.Singkatan.apply(lambda x: x.strip().lower())
dict_singkatan = pd.Series(df_acronym.kepanjangan.values,index=df_acronym.Singkatan).to_dict()

dict_clean = {**dict_singkatan, **dict_slang}

In [13]:
def preprocessing_slang_singkatan(review_text, dict_clean=dict_clean):
  regex = r"\b(?:"+"|".join(re.escape(word) for word in dict_clean) + r")\b"
  reobj = re.compile(regex, re.I)
  
  return reobj.sub(lambda x:dict_clean[x.group(0)], review_text)

In [14]:
def clean_all(x, dict_clean=dict_clean):
  x = preprocessing_slang_singkatan(x, dict_clean=dict_clean)
  return x

In [15]:
tqdm.pandas()

In [16]:
dataset['Clean_text'] = dataset.Clean_text.progress_apply(lambda x: clean_all(x, dict_clean=dict_clean))

100%|██████████| 3960/3960 [00:29<00:00, 132.99it/s]


In [38]:
dataset.to_csv('data_preprocessing_tanpastopword.csv', index=False, sep= ';')

In [39]:
# Select Only Text Column
data1 = dataset[['Clean_text']]
data1

,Clean_text
0,sunscreen mahal yang pernah saya beli ikatan n...
1,ikatan notaris indonesia enak banget pakai ent...
2,the description is quite right produk ikatan n...
3,bisa untuk wajah dan badan dengan harga yang s...
4,saya beli produk ikatan notaris indonesia kare...
...,...
3955,sesal tidak ujung karena sudah beli ikatan not...
3956,buat kulit saya yang oily dehydrated plus sens...
3957,singkat produk ikatan notaris indonesia tekstu...
3958,hai pretty mau kulit kalian fresh elastis dan ...


In [40]:
from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer, CountVectorizer
tfidf_transformer = TfidfTransformer()
tfidf_vectorizer = TfidfVectorizer()
count_vectorize = CountVectorizer()

In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = [
    'Pusing satu kali',
    'Pusing dua kali',
    'Pusing tiga kali',
    'Pusing lagi']

vectorizer = TfidfVectorizer()

# contoh
a = vectorizer.fit_transform(corpus)
a.toarray()

array([[0.        , 0.49248889, 0.        , 0.40264194, 0.77157901,
        0.        ],
       [0.77157901, 0.49248889, 0.        , 0.40264194, 0.        ,
        0.        ],
       [0.        , 0.49248889, 0.        , 0.40264194, 0.        ,
        0.77157901],
       [0.        , 0.        , 0.88654763, 0.46263733, 0.        ,
        0.        ]])

In [42]:
a = dataset['Clean_text']
# Count TF_IDF Vectorizer
count_vectorizer = CountVectorizer()
count_vector = count_vectorizer.fit_transform(dataset['Clean_text'])
count_vector.shape

(3960, 8688)

In [43]:
a = dataset['Clean_text']
tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer = tfidf_vectorizer.fit_transform(a)
print(tfidf_vectorizer)


  (0, 5751)	0.04296347726735528
  (0, 4610)	0.10749404950005755
  (0, 3411)	0.05149451055404851
  (0, 2962)	0.08905596909425262
  (0, 6241)	0.06709019429499166
  (0, 6147)	0.18869292360349904
  (0, 1403)	0.21423350365781993
  (0, 1666)	0.03899671919992412
  (0, 2030)	0.16775772343629813
  (0, 7840)	0.14782318017961746
  (0, 8345)	0.15578522850615034
  (0, 4877)	0.08538613671582695
  (0, 5012)	0.06724036279818824
  (0, 1115)	0.13852305965733744
  (0, 7269)	0.11006815381610201
  (0, 7926)	0.11832981776032271
  (0, 3357)	0.22829469208331607
  (0, 949)	0.2546539995949434
  (0, 5972)	0.2330902962771426
  (0, 1238)	0.15666162020209778
  (0, 8498)	0.14404766238450795
  (0, 7982)	0.13443831943041906
  (0, 7961)	0.13173240583513657
  (0, 7413)	0.12033760133271776
  (0, 3061)	0.19563488553131494
  :	:
  (3959, 3352)	0.07041299180570383
  (3959, 4754)	0.12722545044516648
  (3959, 7100)	0.07586037389782749
  (3959, 7798)	0.11460402797487233
  (3959, 696)	0.16919823591629948
  (3959, 4695)	0.080606

In [44]:
tfidf_vectorizer.shape

(3960, 8688)

In [23]:
vectorizer = TfidfVectorizer(max_features=1000)
vectors = vectorizer.fit_transform(dataset.Clean_text)
words_dataset = pd.DataFrame(vectors.toarray(), columns=vectorizer.get_feature_names())
words_dataset.head()

,aceh,acid,acne,ada,adalah,adat,adem,administrasi,advanced,affordable,...,would,wow,yaa,yah,yaitu,yang,yes,you,your,zone
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.053821,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.056286,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.143208,...,0.0,0.0,0.0,0.0,0.0,0.044601,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.095332,0.0,0.0,0.0,0.0


In [24]:
#seleksi fitur
import numpy as np
from scipy import stats
from sklearn.multiclass import OneVsOneClassifier
from sklearn.multiclass import OutputCodeClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import MultiOutputRegressor,RegressorChain
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multioutput import ClassifierChain
from sklearn import datasets
from sklearn.feature_selection import SelectKBest,chi2 , f_regression, SelectFromModel, SelectPercentile
from sklearn.preprocessing import LabelBinarizer
from sklearn.datasets import load_iris
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import MultiLabelBinarizer

In [25]:
x = tfidf_vectorizer.copy()

In [26]:
y = np.array(dataset[['price','packaging','product','aroma']])

In [27]:
x.shape, y.shape

((3960, 8688), (3960, 4))

In [28]:
Y = MultiLabelBinarizer().fit_transform(y)

In [29]:
x_new = SelectKBest(chi2, k=20).fit_transform(x, Y)

In [30]:
print(x_new)

  (0, 19)	0.12797869038661677
  (0, 10)	0.09180429517205321
  (4, 18)	0.15419283425817942
  (4, 10)	0.08099823262393753
  (5, 2)	0.04731680474655674
  (5, 18)	0.11969739388217833
  (7, 19)	0.18915945893600386
  (7, 10)	0.1356917370406438
  (8, 7)	0.11897836655622848
  (9, 11)	0.08101422058605826
  (9, 19)	0.11930206563444372
  (11, 11)	0.07736629418685625
  (12, 16)	0.14540063132736578
  (12, 10)	0.12708879038168763
  (13, 7)	0.1677323114047679
  (13, 19)	0.18101133234227906
  (13, 10)	0.12984675600005183
  (14, 7)	0.17881755728252746
  (15, 11)	0.17650462862158445
  (16, 9)	0.10647367851560693
  (17, 1)	0.12212003850652406
  (17, 9)	0.05904125405208762
  (17, 7)	0.05968568212023697
  (18, 18)	0.08100198295432323
  (19, 14)	0.07602582338220722
  :	:
  (3948, 2)	0.10892620186145158
  (3949, 18)	0.08238075135902481
  (3949, 10)	0.08655000466675818
  (3950, 5)	0.11640447530018683
  (3950, 0)	0.07924994595137243
  (3950, 4)	0.07924994595137243
  (3950, 2)	0.07296710943514503
  (3951, 14)	0

In [31]:
Kbest = SelectKBest(score_func = chi2, k=20).fit(x, Y)

In [32]:
skor_chi = pd.DataFrame({'features': x_new, 'Chi2Score': Kbest.scores_, 'pValue':Kbest.pvalues_ })
skor_chi

,features,Chi2Score,pValue
0,"(0, 19)\t0.12797869038661677\n (0, 10)\t0.0...",0.083460,0.959129
1,"(0, 19)\t0.12797869038661677\n (0, 10)\t0.0...",0.243127,0.885535
2,"(0, 19)\t0.12797869038661677\n (0, 10)\t0.0...",0.093635,0.954262
3,"(0, 19)\t0.12797869038661677\n (0, 10)\t0.0...",0.286268,0.866638
4,"(0, 19)\t0.12797869038661677\n (0, 10)\t0.0...",0.099662,0.951390
...,...,...,...
8683,"(0, 19)\t0.12797869038661677\n (0, 10)\t0.0...",0.129254,0.937417
8684,"(0, 19)\t0.12797869038661677\n (0, 10)\t0.0...",0.080860,0.960376
8685,"(0, 19)\t0.12797869038661677\n (0, 10)\t0.0...",0.157840,0.924114
8686,"(0, 19)\t0.12797869038661677\n (0, 10)\t0.0...",0.023798,0.988171


In [33]:
b = skor_chi[['Chi2Score']]
b

,Chi2Score
0,0.083460
1,0.243127
2,0.093635
3,0.286268
4,0.099662
...,...
8683,0.129254
8684,0.080860
8685,0.157840
8686,0.023798


In [34]:
from sklearn.model_selection import train_test_split
x_new_train, x_new_test, Y_train, Y_test = train_test_split(x_new,Y, test_size=0.2, random_state=0)

In [35]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor


knn = KNeighborsClassifier(n_neighbors=11)
## Fit the model on the training data.
classifier = MultiOutputClassifier(knn, n_jobs=-1)
classifier.fit(x_new_train, Y_train)
## See how the model performs on the test data.
predictions = classifier.predict(x_new_test)

In [36]:
knn_skor=classifier.score(x_new_test,np.array(Y_test))
print(knn_skor * 100 ,'%')

68.81313131313132 %


In [37]:
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import recall_score
from sklearn.metrics import  multilabel_confusion_matrix

multilabel = multilabel_confusion_matrix(Y_test,predictions)
f1_score = metrics.f1_score(Y_test,predictions, average='micro', labels=np.unique(predictions))
precision_score = metrics.precision_score(Y_test,predictions, average='micro', labels=np.unique(predictions))
recall_score = metrics.recall_score(Y_test,predictions, average='micro', labels=np.unique(predictions))
        
print('akurasi f1 score        :', f1_score* 100 ,'%')
print('akurasi precision score :', precision_score* 100 ,'%')
print('akurasi recall score    :', recall_score* 100 ,'%')

akurasi f1 score        : 91.53664302600473 %
akurasi precision score : 95.2755905511811 %
akurasi recall score    : 88.0800727934486 %
